In [624]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import random
import clonalg
import pandas as pd
from sklearn import preprocessing

In [687]:
df = pd.read_csv('dataset/iris.csv', header=None)

df[4] = pd.Categorical(df[4]).codes

df = df.loc[df[4] == 0]

scaler = preprocessing.MinMaxScaler()
train = pd.DataFrame(scaler.fit_transform(df.values), columns=df.columns, index=df.index)

labels = train[[4]].copy()
train = train.drop(columns=[4])

In [641]:
# Inputs parameters
feature_num = len(train.axes[1])
feature_max = train.max()
feature_max  = feature_max.max()
feature_min = train.min()
feature_min  = feature_min.min()

threshold = 0.1

population_size = 300
selection_size = 20
random_cells_num = 20
clone_rate = 50
mutation_rate = 0.3
stop_codition = 10000

AttributeError: 'numpy.ndarray' object has no attribute 'axes'

In [642]:
train = train.to_numpy()

AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [ ]:
stop = 0
population = clonalg.create_random_cells(population_size, feature_num, feature_min, feature_max)
best_affinity_it = []

while stop != stop_codition:
    antigen = train[np.random.randint(0, len(train))]
    population_affinity = [(cell, clonalg.affinity(cell, antigen)) for cell in population]
    populatin_affinity = sorted(population_affinity, key=lambda x: x[1])

    # Número de melhores individuos delecionados
    best_affinity_it.append(populatin_affinity[:10])

    population_select = populatin_affinity[:selection_size]

    population_clones = []
    for cell in population_select:
        cell_clones = clonalg.clone(cell, clone_rate)
        population_clones += cell_clones

    pop_clones_tmp = []
    for cell in population_clones:
        ind_tmp = clonalg.hypermutate(cell, mutation_rate, feature_min, feature_max, antigen)
        pop_clones_tmp.append(ind_tmp)
    population_clones = pop_clones_tmp
    del pop_clones_tmp

    population = clonalg.select(populatin_affinity, population_clones, population_size)

    population_rand = clonalg.create_random_cells(random_cells_num, feature_num, feature_min, feature_max)
    population_rand_affinity = [(cell, clonalg.affinity(cell, antigen)) for cell in population_rand]
    population_rand_affinity = sorted(population_rand_affinity, key=lambda x: x[1])

    population = clonalg.replace(population_affinity, population_rand_affinity, population_size)
    population = [cell[0] for cell in population]
    
    stop += 1

In [ ]:
bests_mean = []
iterations = [i for i in range(10000)]

for pop_it in best_affinity_it:
    bests_mean.append(np.mean([cell[1] for cell in pop_it]))

In [ ]:
pop_it

In [679]:
for i in range(len(pop_it)):
    population[i] = pop_it[i][0]

In [688]:
antibodies = pd.DataFrame(population)
antibodies = pd.concat([antibodies, labels], axis=1, join="inner")
antibodies = pd.DataFrame(scaler.inverse_transform(antibodies), columns=antibodies.columns)
antibodies = antibodies.drop(columns=[4])

In [689]:
antigens = df.head(10)
antigens = antigens.drop(columns=[4])
antigens = antigens.to_numpy()
antibodies = antibodies.head(10).to_numpy()

In [690]:
aff = []
aff_antigen = []
for i in range(len(antibodies)):
    for j in range(len(antibodies)):
         aff_antigen.append((abs(affinity(antibodies[i], antigens[j]))))
    aff.append(min(aff_antigen))

In [691]:
aff

[1.0480947928079645,
 1.0480947928079645,
 0.9574852881871104,
 0.9574852881871104,
 0.912207858727249,
 0.912207858727249,
 0.912207858727249,
 0.912207858727249,
 0.912207858727249,
 0.912207858727249]

In [695]:
for i in range(len(aff)):
    if aff[i] < threshold:
        print('The antigen ', i, ' reacted to the antibody')

In [673]:
plot_df = pd.DataFrame(antibodies, columns = ['A','B','C','D'])